Hello!

For many R users, This kernel has been converted from Python to R.

I referenced this kernel. https://www.kaggle.com/shanth84/home-credit-bureau-data-feature-engineering

I am still not enough, but I will try hard.

# HOW TO INTERPRET BUREAU DATA
This table talks about the Loan data of each unique customer with all financial institutions other than Home Credit For each unique SK_ID_CURR we have multiple SK_ID_BUREAU Id's, each being a unique loan transaction from other financial institutions availed by the same customer and reported to the bureau.

In [1]:
library(data.table) # Fast CSV read
library(tidyverse)    #Visualisation
library(caret) #Cross vaildation
library(xgboost) #Xgboost
library(lightgbm) #Lightgbm
library(DataExplorer)
library(knitr)
library(gridExtra)
library(scales)
library(reshape2)
library(magrittr)
library(repr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.1          ✔ dplyr   0.8.1     
✔ tidyr   0.8.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()
Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

The following object is masked from ‘package:httr’:

    progress


Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice

Loading required package: R6

Attaching package: ‘lightgbm’

The following objects are m

In [2]:
bbalance <- fread("../input/bureau_balance.csv", na.strings=c("NA","NaN","?", "","XNA"))
bureau <- fread("../input/bureau.csv", na.strings=c("NA","NaN","?", "","XNA"))
# credit <- fread("../input/credit_card_balance.csv", na.strings=c("NA","NaN","?", "","XNA"))
# install <- fread("../input/installments_payments.csv", na.strings=c("NA","NaN","?", "","XNA")) 
# pos <- fread("../input/POS_CASH_balance.csv", na.strings=c("NA","NaN","?", "","XNA"))
# previous <- fread("../input/previous_application.csv", na.strings=c("NA","NaN","?", "","XNA"))
train <- fread("../input/application_train.csv", na.strings=c("NA","NaN","?", "","XNA")) 
test <- fread("../input/application_test.csv", na.strings=c("NA","NaN","?", "","XNA"))

In [3]:
bureau = bureau  %>%
        mutate_if(is.integer, list(~as.numeric(.)))  %>%
        mutate_if(is.character, list(~as.factor(.)))

bbalance = bbalance  %>%
        mutate_if(is.integer, list(~as.numeric(.)))  %>% 
        mutate_if(is.character, list(~as.factor(.)))

In [4]:
cat("application_train : (" , nrow(bbalance) , "," , ncol(bbalance) , ")\n")
cat("application_test : (" , nrow(bureau) , "," , ncol(bureau) , ")")

application_train : ( 27299925 , 3 )
application_test : ( 1716428 , 17 )

# EXAMPLE OF BUREAU TRANSACTIONS
* In the example below customer with SK_ID_CURR = 100001 had 7 credit transactions before the current application

In [5]:
bureau  %>% filter(SK_ID_CURR==100001)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
100001,5896630,Closed,currency 1,-857,0,-492,-553,NA,0,112500,0.0,0,0,Consumer credit,-155,0.0
100001,5896631,Closed,currency 1,-909,0,-179,-877,NA,0,279720,0.0,0,0,Consumer credit,-155,0.0
100001,5896632,Closed,currency 1,-879,0,-514,-544,NA,0,91620,0.0,0,0,Consumer credit,-155,0.0
100001,5896633,Closed,currency 1,-1572,0,-1329,-1328,NA,0,85500,0.0,0,0,Consumer credit,-155,0.0
100001,5896634,Active,currency 1,-559,0,902,NA,NA,0,337680,113166.0,0,0,Consumer credit,-6,4630.5
100001,5896635,Active,currency 1,-49,0,1778,NA,NA,0,378000,373239.0,0,0,Consumer credit,-16,10822.5
100001,5896636,Active,currency 1,-320,0,411,NA,NA,0,168345,110281.5,NA,0,Consumer credit,-10,9364.5


# UNDERSTANDING OF VARIABLES

CREDIT_ACTIVE - Current status of a Loan - Closed/ Active (2 values)

CREDIT_CURRENCY - Currency in which the transaction was executed - Currency1, Currency2, Currency3, Currency4 ( 4 values)

CREDIT_DAY_OVERDUE - Number of overdue days

CREDIT_TYPE - Consumer Credit, Credit card, Mortgage, Car loan, Microloan, Loan for working capital replemishment, Loan for Business development, Real estate loan, Unkown type of laon, Another type of loan. Cash loan, Loan for the purchase of equipment, Mobile operator loan, Interbank credit, Loan for purchase of shares ( 15 values )

DAYS_CREDIT - Number of days ELAPSED since customer applied for CB credit with respect to current application Interpretation - Are these loans evenly spaced time intervals? Are they concentrated within a same time frame?

DAYS_CREDIT_ENDDATE - Number of days the customer CREDIT is valid at the time of application CREDIT_DAY_OVERDUE - Number of days the customer CREDIT is past the end date at the time of application

AMT_CREDIT_SUM - Total available credit for a customer AMT_CREDIT_SUM_DEBT - Total amount yet to be repayed AMT_CREDIT_SUM_LIMIT - Current Credit that has been utilized AMT_CREDIT_SUM_OVERDUE - Current credit payment that is overdue CNT_CREDIT_PROLONG - How many times was the Credit date prolonged

# NOTE:
For a given loan transaction 'AMT_CREDIT_SUM' = 'AMT_CREDIT_SUM_DEBT' +'AMT_CREDIT_SUM_LIMIT'

* AMT_ANNUITY - Annuity of the Credit Bureau data 
* DAYS_CREDIT_UPDATE - Number of days before current application when last CREDIT UPDATE was received 
* DAYS_ENDDATE_FACT - Days since CB credit ended at the time of application 
* AMT_CREDIT_MAX_OVERDUE - Maximum Credit amount overdue at the time of application

# FEATURE ENGINEERING WITH BUREAU CREDIT

# FEATURE 1 - NUMBER OF PAST LOANS PER CUSTOMER

In [6]:
B=bureau[1:10000,]

grp=B[,c('SK_ID_CURR', 'DAYS_CREDIT')]  %>% 
    group_by(SK_ID_CURR)  %>% 
    summarise_all(n_distinct) %>%
    rename(BUREAU_LOAN_COUNT=DAYS_CREDIT)

In [7]:
head(grp)

SK_ID_CURR,BUREAU_LOAN_COUNT
<dbl>,<int>
100053,7
100568,7
100653,7
100802,1
100819,18
100915,1


In [8]:
B=left_join(B,grp,by=c("SK_ID_CURR"))
dim(B)

[1] 10000    18

# FEATURE 2 - NUMBER OF TYPES OF PAST LOANS PER CUSTOMER

In [9]:
B=bureau[1:10000,]

grp=B[,c('SK_ID_CURR', 'CREDIT_TYPE')]  %>% 
    group_by(SK_ID_CURR)  %>% 
    summarise_all(n_distinct) %>%
    rename(BUREAU_LOAN_TYPES=CREDIT_TYPE)

In [10]:
head(grp)

SK_ID_CURR,BUREAU_LOAN_TYPES
<dbl>,<int>
100053,2
100568,2
100653,2
100802,1
100819,2
100915,1


In [11]:
B=left_join(B,grp,by=c("SK_ID_CURR"))
dim(B)

[1] 10000    18

# FEATURE 3 - AVERAGE NUMBER OF PAST LOANS PER TYPE PER CUSTOMER

## Is the Customer diversified in taking multiple types of Loan or Focused on a single type of loan

In [12]:
B=bureau[1:10000,]

# Number of Loans per Customer
grp=B[,c('SK_ID_CURR', 'DAYS_CREDIT')]  %>% 
    group_by(SK_ID_CURR)  %>% 
    summarise_all(n_distinct) %>%
    rename(BUREAU_LOAN_COUNT=DAYS_CREDIT)

B=left_join(B,grp,by=c("SK_ID_CURR"))


# Number of types of Credit loans for each Customer 
grp=B[,c('SK_ID_CURR', 'CREDIT_TYPE')]  %>% 
    group_by(SK_ID_CURR)  %>% 
    summarise_all(n_distinct) %>%
    rename(BUREAU_LOAN_TYPES=CREDIT_TYPE)

B=left_join(B,grp,by=c("SK_ID_CURR"))

# Average Number of Loans per Loan Type
B=B  %>% 
    mutate(AVERAGE_LOAN_TYPE = BUREAU_LOAN_COUNT / BUREAU_LOAN_TYPES) %>% 
    select(-c(BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES))

dim(B)
gc()

[1] 10000    18

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2241689,119.8,3856742,206.0,3856742,206.0
Vcells,165122812,1259.8,289364718,2207.7,280342913,2138.9


# FEATURE 4 - % OF ACTIVE LOANS FROM BUREAU DATA

In [13]:
B = bureau[1:10000,]
# Create a new dummy column for whether CREDIT is ACTIVE OR CLOED 
B[,'CREDIT_ACTIVE_BINARY'] = ifelse(B$'CREDIT_ACTIVE'=='Closed',0,1)

In [14]:
# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp=B[,c('SK_ID_CURR', 'CREDIT_ACTIVE_BINARY')] %>% 
    group_by(SK_ID_CURR)  %>% 
    summarise_all(mean) %>%
    rename(ACTIVE_LOANS_PERCENTAGE = CREDIT_ACTIVE_BINARY)

In [15]:
B=left_join(B,grp,by=c("SK_ID_CURR"))  %>% select(-CREDIT_ACTIVE_BINARY)
dim(B)
gc()

[1] 10000    18

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2242611,119.8,3856742,206.0,3856742,206.0
Vcells,165122707,1259.8,289364718,2207.7,280342913,2138.9


In [16]:
B  %>% filter(SK_ID_CURR==100653)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,ACTIVE_LOANS_PERCENTAGE
<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
100653,5726282,Active,currency 1,-256,0,109,NA,NA,0,97244.91,39081.42,NA,0,Consumer credit,-37,0,0.5714286
100653,5726283,Closed,currency 1,-2474,0,-2260,-2289,NA,0,30262.14,0.00,NA,0,Consumer credit,-856,NA,0.5714286
100653,5726284,Closed,currency 1,-2258,0,-1924,-1924,NA,0,47156.85,0.00,NA,0,Consumer credit,-856,0,0.5714286
100653,5726285,Closed,currency 1,-1677,0,-946,-946,NA,0,71068.50,0.00,0,0,Consumer credit,-895,0,0.5714286
100653,5726286,Active,currency 1,-182,0,-152,NA,NA,0,225000.00,215923.50,0,0,Credit card,-82,0,0.5714286
100653,5726287,Active,currency 1,-1564,0,NA,NA,NA,0,87750.00,0.00,NA,0,Credit card,-1006,0,0.5714286
100653,5726288,Active,currency 1,-252,0,3401,NA,NA,0,315000.00,201766.50,NA,0,Credit card,-22,NA,0.5714286


# FEATURE 5

* AVERAGE NUMBER OF DAYS BETWEEN SUCCESSIVE PAST APPLICATIONS FOR EACH CUSTOMER¶

* How often did the customer take credit in the past? Was it spaced out at regular time intervals - a signal of good financial planning OR were the loans concentrated around a smaller time frame - indicating potential financial trouble?

In [17]:
B = bureau[1:10000,]

# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
grp=B[,c('SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT')] %>% 
    group_by(SK_ID_CURR)  %>% 
    arrange(desc(DAYS_CREDIT))

In [18]:
# Calculate Difference between the number of Days 
grp[,'DAYS_CREDIT1'] = grp[,'DAYS_CREDIT']*-1
 
grp=grp %>%
    group_by(SK_ID_CURR) %>%
    mutate(DAYS_DIFF = DAYS_CREDIT1 - lag(DAYS_CREDIT1)) %>% 
    ungroup()

In [19]:
grp$DAYS_DIFF=ifelse(is.na(grp$DAYS_DIFF),0,grp$DAYS_DIFF)

In [20]:
grp = grp %>% 
        select(-c(SK_ID_CURR,DAYS_CREDIT1,DAYS_CREDIT))

In [21]:
head(grp)

SK_ID_BUREAU,DAYS_DIFF
<dbl>,<dbl>
5717955,0
5715435,0
5715444,3
5721198,0
5722488,0
5721317,0


In [22]:
B=left_join(B,grp,by=c("SK_ID_BUREAU"))
dim(B)
gc()

[1] 10000    18

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2244586,119.9,3856742,206.0,3856742,206.0
Vcells,165143151,1260.0,289364718,2207.7,280342913,2138.9


# FEATURE 6

* % of LOANS PER CUSTOMER WHERE END DATE FOR CREDIT IS PAST

* INTERPRETING CREDIT_DAYS_ENDDATE

* NEGATIVE VALUE - Credit date was in the past at time of application( Potential Red Flag !!! )

* POSITIVE VALUE - Credit date is in the future at time of application ( Potential Good Sign !!!!)

* NOTE : This is not the same as % of Active loans since Active loans

* can have Negative and Positive values for DAYS_CREDIT_ENDDATE

In [23]:
B = bureau[1:10000,]
B[,'CREDIT_ENDDATE_BINARY'] = B[,'DAYS_CREDIT_ENDDATE']

In [24]:
B$'CREDIT_ENDDATE_BINARY'=ifelse(B$'CREDIT_ENDDATE_BINARY'<0,0,1)

In [25]:
grp=B %>%
    group_by(SK_ID_CURR) %>%
    summarise(CREDIT_ENDDATE_PERCENTAGE = mean(CREDIT_ENDDATE_BINARY,na.rm=T)) %>% 
    ungroup()

In [26]:
head(grp,10)

SK_ID_CURR,CREDIT_ENDDATE_PERCENTAGE
<dbl>,<dbl>
100053,0.0000000
100568,0.3333333
100653,0.3333333
100802,1.0000000
100819,0.2777778
100915,1.0000000
101060,0.5000000
101133,0.0000000
101562,0.6000000


In [27]:
B=left_join(B,grp,by=c("SK_ID_CURR"))  %>% select(-CREDIT_ENDDATE_BINARY)
dim(B)
gc()

[1] 10000    18

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2244863,119.9,3856742,206.0,3856742,206.0
Vcells,165127934,1259.9,289364718,2207.7,280342913,2138.9


# FEATURE 7

* AVERAGE NUMBER OF DAYS IN WHICH CREDIT EXPIRES IN FUTURE -INDICATION OF CUSTOMER DELINQUENCY IN FUTURE??

In [28]:
# Repeating Feature 6 to Calculate all transactions with ENDATE as POSITIVE VALUES 
B = bureau[1:10000,]
B[,'CREDIT_ENDDATE_BINARY'] = B[,'DAYS_CREDIT_ENDDATE']

B$'CREDIT_ENDDATE_BINARY'=ifelse(B$'CREDIT_ENDDATE_BINARY'<0,0,1)

In [29]:
# We take only positive values of  ENDDATE since we are looking at Bureau Credit VALID IN FUTURE 
# as of the date of the customer's loan application with Home Credit 

B1 = B  %>% filter(CREDIT_ENDDATE_BINARY== 1)
dim(B1)

[1] 3551   18

In [30]:
#Calculate Difference in successive future end dates of CREDIT 

# Create Dummy Column for CREDIT_ENDDATE 
B1[,'DAYS_CREDIT_ENDDATE1'] = B1[,'DAYS_CREDIT_ENDDATE']

# Groupby Each Customer ID 
# Sort the values of CREDIT_ENDDATE for each customer ID 
grp = B1[,c('SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE1')] %>% 
        group_by(SK_ID_CURR)   %>% 
        arrange(desc(DAYS_CREDIT_ENDDATE1))

In [31]:
# Calculate the Difference in ENDDATES and fill missing values with zero 
grp=grp %>%
    group_by(SK_ID_CURR) %>%
    mutate(DAYS_ENDDATE_DIFF = lag(DAYS_CREDIT_ENDDATE1) - DAYS_CREDIT_ENDDATE1) %>% 
    ungroup()

In [32]:
grp$DAYS_ENDDATE_DIFF = grp$DAYS_ENDDATE_DIFF  %>% replace_na(0)

In [33]:
grp = grp  %>% 
        select(-c(DAYS_CREDIT_ENDDATE1,SK_ID_CURR))

In [34]:
# Merge new feature 'DAYS_ENDDATE_DIFF' with original Data frame for BUREAU DATA
B=left_join(B,grp,by=c("SK_ID_BUREAU"))
dim(B)

[1] 10000    19

In [35]:
grp = B[,c('SK_ID_CURR', 'DAYS_ENDDATE_DIFF')] %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(mean,na.rm=T) %>% 
        rename(AVG_ENDDATE_FUTURE = DAYS_ENDDATE_DIFF)

In [36]:
head(grp)

SK_ID_CURR,AVG_ENDDATE_FUTURE
<dbl>,<dbl>
100053,NaN
100568,138.5
100653,1646.0
100802,0.0
100819,241.0
100915,0.0


In [37]:
B=left_join(B,grp,by=c("SK_ID_CURR"))  %>% select(-c(CREDIT_ENDDATE_BINARY,DAYS_CREDIT_ENDDATE))
dim(B)
gc()

[1] 10000    18

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2245489,120.0,3856742,206.0,3856742,206.0
Vcells,165187845,1260.3,289364718,2207.7,280342913,2138.9


In [38]:
B  %>% filter(SK_ID_CURR==100653)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_ENDDATE_DIFF,AVG_ENDDATE_FUTURE
<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
100653,5726282,Active,currency 1,-256,0,NA,NA,0,97244.91,39081.42,NA,0,Consumer credit,-37,0,3292,1646
100653,5726283,Closed,currency 1,-2474,0,-2289,NA,0,30262.14,0.00,NA,0,Consumer credit,-856,NA,NA,1646
100653,5726284,Closed,currency 1,-2258,0,-1924,NA,0,47156.85,0.00,NA,0,Consumer credit,-856,0,NA,1646
100653,5726285,Closed,currency 1,-1677,0,-946,NA,0,71068.50,0.00,0,0,Consumer credit,-895,0,NA,1646
100653,5726286,Active,currency 1,-182,0,NA,NA,0,225000.00,215923.50,0,0,Credit card,-82,0,NA,1646
100653,5726287,Active,currency 1,-1564,0,NA,NA,0,87750.00,0.00,NA,0,Credit card,-1006,0,NA,1646
100653,5726288,Active,currency 1,-252,0,NA,NA,0,315000.00,201766.50,NA,0,Credit card,-22,NA,0,1646


# FEATURE 8

* The Ratio of Total Debt to Total Credit for each Customer

* A High value may be a red flag indicative of potential default

In [39]:
# AMT_CREDIT_SUM = AMT_CREDIT_SUM_DEBT + AMT_CREDIT_SUM_LIMIT

In [40]:
B = bureau[1:10000,]
B[,'AMT_CREDIT_SUM_DEBT'] = B[,'AMT_CREDIT_SUM_DEBT']  %>% replace_na(0)

B[,'AMT_CREDIT_SUM']=B[,'AMT_CREDIT_SUM']  %>% replace_na(0)

In [41]:
grp1=B[,c('SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT')]  %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(sum,na.rm=T) %>%
        rename(TOTAL_CUSTOMER_DEBT = AMT_CREDIT_SUM_DEBT)


grp2=B[,c('SK_ID_CURR', 'AMT_CREDIT_SUM')]  %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(sum,na.rm=T) %>%
        rename(TOTAL_CUSTOMER_CREDIT =AMT_CREDIT_SUM)

In [42]:
head(grp1)

SK_ID_CURR,TOTAL_CUSTOMER_DEBT
<dbl>,<dbl>
100053,0.0
100568,422599.5
100653,456771.4
100802,157630.5
100819,1403749.9
100915,4.5


In [43]:
B=left_join(B,grp1,by=c("SK_ID_CURR")) 
B=left_join(B,grp2,by=c("SK_ID_CURR")) 
rm(grp1,grp2)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2245468,120.0,3856742,206.0,3856742,206.0
Vcells,165197832,1260.4,289364718,2207.7,280342913,2138.9


In [44]:
B= B %>% 
    mutate(DEBT_CREDIT_RATIO = TOTAL_CUSTOMER_DEBT / TOTAL_CUSTOMER_CREDIT)  %>% 
    select(-c(TOTAL_CUSTOMER_DEBT,TOTAL_CUSTOMER_CREDIT))

In [45]:
dim(B)

[1] 10000    18

# FEATURE 9 - OVERDUE OVER DEBT RATIO

* What fraction of total Debt is overdue per customer?¶

* A high value could indicate a potential DEFAULT

In [46]:
B = bureau[1:10000,]
B[,'AMT_CREDIT_SUM_DEBT'] = B[,'AMT_CREDIT_SUM_DEBT']  %>% replace_na(0)

B[,'AMT_CREDIT_SUM']=B[,'AMT_CREDIT_SUM']  %>% replace_na(0)

In [47]:
grp1=B[,c('SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT')]  %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(sum,na.rm=T) %>%
        rename(TOTAL_CUSTOMER_DEBT = AMT_CREDIT_SUM_DEBT)


grp2=B[,c('SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE')]  %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(sum,na.rm=T) %>%
        rename(TOTAL_CUSTOMER_OVERDUE =AMT_CREDIT_SUM_OVERDUE)

In [48]:
B=left_join(B,grp1,by=c("SK_ID_CURR")) 
B=left_join(B,grp2,by=c("SK_ID_CURR")) 
rm(grp1,grp2)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2245465,120.0,3856742,206.0,3856742,206.0
Vcells,165197825,1260.4,289364718,2207.7,280342913,2138.9


In [49]:
B= B %>% 
    mutate(OVERDUE_DEBT_RATIO = TOTAL_CUSTOMER_OVERDUE / TOTAL_CUSTOMER_DEBT)  %>% 
    select(-c(TOTAL_CUSTOMER_DEBT,TOTAL_CUSTOMER_OVERDUE))

In [50]:
dim(B)

[1] 10000    18

# FEATURE 10 - AVERAGE NUMBER OF LOANS PROLONGED

In [51]:
B = bureau[1:10000,]
B[,'CNT_CREDIT_PROLONG'] = B[,'CNT_CREDIT_PROLONG']  %>% replace_na(0)

In [52]:
grp1=B[,c('SK_ID_CURR', 'CNT_CREDIT_PROLONG')]  %>% 
        group_by(SK_ID_CURR) %>% 
        summarise_all(mean,na.rm=T) %>%
        rename(AVG_CREDITDAYS_PROLONGED = CNT_CREDIT_PROLONG)

In [53]:
head(grp1)

SK_ID_CURR,AVG_CREDITDAYS_PROLONGED
<dbl>,<dbl>
100053,0
100568,0
100653,0
100802,0
100819,0
100915,0


In [54]:
B=left_join(B,grp1,by=c("SK_ID_CURR")) 

In [55]:
dim(B)

[1] 10000    18